In [ ]:
import os
import nltk
import pandas as pd
import numpy as np
curentdirectory='/content/drive/MyDrive/Distilbert_model/'
os.chdir(curentdirectory)

In [ ]:
# !pip install torch==1.7.1
# !pip install transformers==3.5.1
import tensorflow as tf
import pandas as pd
import json
import gc
import os
import numpy as np
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification

In [ ]:
BERT_PATH = "distilbert-base-uncased"
MODEL_PATH = "DistilBert_modelh.h5"
TRAINING_FILE = "Cleaned_Training_Testing_Mapping_Files/train_iris_fullcleaned.xlsx"
DistilBert_tokenizer = DistilBertTokenizer.from_pretrained(BERT_PATH, do_lower_case=True)

df=pd.read_excel(TRAINING_FILE)
num_classes=len(df.intent1.unique())
d= dict([(i,a) for i, a in zip(df.intent1, df.intent)])

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)
model_save_path='DistilBert_modelh.h5'
trained_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=num_classes)
trained_model.compile(loss=loss,optimizer=optimizer, metrics=[metric])
trained_model.load_weights(model_save_path)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'vocab_projector', 'vocab_layer_norm', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
df = pd.read_excel('Cleaned_Training_Testing_Mapping_Files/test_iris_fullcleaned.xlsx') # Change path to your download location
num_classes=len(df.intent1.unique())
sentences=df['utterance']
labels=df['intent1']
input_ids=[]
attention_masks=[]
for sent in sentences:
    DistilBert_inp=DistilBert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =64,pad_to_max_length = True,return_attention_mask = True)
    input_ids.append(DistilBert_inp['input_ids'])
    attention_masks.append(DistilBert_inp['attention_mask'])
input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
preds = trained_model.predict([input_ids,attention_masks],batch_size=32)

In [ ]:
pred_labels = tf.argmax(preds[0],axis=1)
pred_prob = tf.nn.softmax(tf.convert_to_tensor(preds[0]))
pred_prob_top,classes = tf.nn.top_k(pred_prob, 4)
top_prob1=pred_prob_top[0][0]
top_prob2=pred_prob_top[0][1]
top_prob3=pred_prob_top[0][2]
top_prob4=pred_prob_top[0][3]
top_class1=classes[0][0]
top_class2=classes[0][1]
top_class3=classes[0][2]
top_class4=classes[0][3]
print(top_prob1,top_prob2,top_prob3,top_class1,top_class2,top_class3)
# pred_labels=tf.argmax(preds[0][0], axis=1)
pred_labels = tf.argmax(preds[0],axis=1)
print(pred_labels)

tf.Tensor(0.99115527, shape=(), dtype=float32) tf.Tensor(0.0011708471, shape=(), dtype=float32) tf.Tensor(0.0007248843, shape=(), dtype=float32) tf.Tensor(4, shape=(), dtype=int32) tf.Tensor(335, shape=(), dtype=int32) tf.Tensor(16, shape=(), dtype=int32)
tf.Tensor([  4   4  24 ...  76  76 218], shape=(3492,), dtype=int64)


In [ ]:
df["pred_idx"]=pred_labels.numpy()
df["pred_values"]=d[int(top_class1)]
df["top_p1"]=top_prob1.numpy()
df["top_p2"]=top_prob2.numpy()
df["top_p3"]=top_prob3.numpy()
df["top_p4"]=top_prob4.numpy()
df["top_class1"]=top_class1.numpy()
df["top_class2"]=top_class2.numpy()
df["top_class3"]=top_class3.numpy()
df["top_class4"]=top_class4.numpy()
df["predictions1"]=d[int(top_class1)]
df["predictions2"]=d[int(top_class2)]
df["predictions3"]=d[int(top_class3)]
df["predictions4"]=d[int(top_class4)]
df.to_csv('IRIS_predictions_TF.csv',index=None)
from sklearn.metrics import accuracy_score
accuracy_score(df['intent1'],df['pred_idx'])

0.8115693012600229